<a href="https://colab.research.google.com/github/dssg/triage/blob/kit_colab_triage/example/colab/colab_triage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Triage


## Getting Set Up

In [ ]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `donors_choose` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS donors_choose;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE donors_choose;'

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 10.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package cron.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../0-cron_3.0pl1-128.1ubuntu1_amd64.deb ...
Unpacking cron (3.0pl1-128.1ubuntu1) ...
Selecting previously unselected package logrotate.
Preparing to unpack .../1-logrotate_3.11.0-0.1ubuntu1_amd64.deb ...
Unpacking logrotate (3.11.0-0.1ubuntu1) ...
Selecting previously unselected package netbase.
Preparing to unpack .../2-netbase_5.4_all.deb ...
Unpacking netbase (5.4) ...
Selecting previously unselected pac

In [ ]:
%env DEMO_DATABASE_NAME=donors_choose
%env DEMO_DATABASE_HOST=localhost
%env DEMO_DATABASE_PORT=5432
%env DEMO_DATABASE_USER=postgres
%env DEMO_DATABASE_PASS=postgres

env: DEMO_DATABASE_NAME=donors_choose
env: DEMO_DATABASE_HOST=localhost
env: DEMO_DATABASE_PORT=5432
env: DEMO_DATABASE_USER=postgres
env: DEMO_DATABASE_PASS=postgres


In [ ]:
!curl -s -OL https://dsapp-public-data-migrated.s3.us-west-2.amazonaws.com/donors_sampled_20210920_v2.dmp

In [ ]:
!ls

donors_sampled_20210920_v2.dmp	sample_data


In [ ]:
!PGPASSWORD=$DEMO_DATABASE_PASS pg_restore -h $DEMO_DATABASE_HOST -p $DEMO_DATABASE_PORT -d $DEMO_DATABASE_NAME -U $DEMO_DATABASE_USER -O -j 8 donors_sampled_20210920_v2.dmp

In [ ]:
!sudo -u postgres psql -U postgres -d donors_choose -c 'ALTER SCHEMA sampled_new RENAME TO data;'

ALTER SCHEMA


In [ ]:
!pip install triage

     |████████████████████████████████| 249 kB 4.1 MB/s 
     |████████████████████████████████| 2.2 MB 45.6 MB/s 
     |████████████████████████████████| 636 kB 51.3 MB/s 
     |████████████████████████████████| 227 kB 41.4 MB/s 
     |████████████████████████████████| 14.6 MB 67 kB/s 
     |████████████████████████████████| 61 kB 417 kB/s 
     |████████████████████████████████| 40 kB 5.4 MB/s 
     |████████████████████████████████| 25.9 MB 94 kB/s 
     |████████████████████████████████| 10.1 MB 33.9 MB/s 
     |████████████████████████████████| 1.1 MB 17.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.3 MB 44.7 MB/s 
     |████████████████████████████████| 2.9 MB 35.0 MB/s 
     |████████████████████████████████| 129 kB 51.9 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 215 kB 58.5 MB/s 
     |██████

## experiment_config.yaml

In [ ]:
config_yaml = """
config_version: 'v7'

model_comment: 'triage demo'

random_seed: 1995

temporal_config:

    # first date our feature data is good
    feature_start_time: '2000-01-01'
    feature_end_time: '2013-06-01'

    # first date our label data is good
    # donorschoose: as far back as we have good donation data
    label_start_time: '2011-09-02'
    label_end_time: '2013-06-01'

    model_update_frequency: '4month'

    # length of time defining a test set
    test_durations: ['3month']
    # defines how far back a training set reaches
    max_training_histories: ['1y']

    # we sample every day, since new projects are posted
    # every day
    training_as_of_date_frequencies: ['1day']
    test_as_of_date_frequencies: ['1day']
    
    # when posted project timeout
    label_timespans: ['3month']
    

cohort_config:
  query: |
    SELECT distinct(entity_id), date_posted as as_of_date
      FROM data.projects
    WHERE date_posted = '{as_of_date}'::date - interval '1day'

label_config:
  query: |
    SELECT entity_id,
    CASE WHEN d.entity_id IS NULL THEN 1 ELSE 0 END AS outcome  
    FROM data.projects
    LEFT JOIN (SELECT DISTINCT entity_id FROM data.donations) d using(entity_id)
    WHERE date_posted BETWEEN '{as_of_date}'::date - interval '1day'
    AND '{as_of_date}'::date + interval '{label_timespan}'

  name: 'any_donations'


feature_aggregations:
  -
    prefix: 'project_features'
    from_obj: 'data.projects'
    knowledge_date_column: 'date_posted'

    aggregates_imputation:
      all:
        type: 'zero'

    categoricals_imputation:
      all:
        type: 'null_category'          

    categoricals:
      -
        column: 'resource_type'
        metrics:
          - 'max' 
        choice_query: 'select distinct resource_type from data.projects'
    
    aggregates:
      -
        quantity: 'total_asking_price'
        metrics:
          - 'sum'
      
    # Since our time-aggregate features are precomputed, feature interval is 
    # irrelvant. We keep 'all' as a default.
    intervals: ['all'] 
    groups: ['entity_id']

grid_config:
    'sklearn.ensemble.RandomForestClassifier':
      n_estimators: [150]
      max_depth: [50]
      min_samples_split: [25]
    
    'sklearn.tree.DecisionTreeClassifier':
      max_depth: [3]
      max_features: [null]
      min_samples_split: [25]
      
    'triage.component.catwalk.estimators.classifiers.ScaledLogisticRegression':
        C: [0.1]
        penalty: ['l1']
    
    'triage.component.catwalk.baselines.rankers.PercentileRankOneFeature':
      feature: ['project_features_entity_id_all_total_asking_price_sum']
      descend: [False]


scoring:
    testing_metric_groups:
        -
          metrics: [precision@, recall@]
          thresholds:
              percentiles: [1, 2, 3, 4, 5, 6, 7, 8, 9, 
                  10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                  20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 
                  30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 
                  40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                  50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
                  60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
                  70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
                  80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
                  90, 91, 92, 93, 94, 95, 96, 97, 98, 99,
                  100]
              top_n: [25, 50, 100]

    training_metric_groups:
        -
          metrics: [precision@, recall@]
          thresholds:
              percentiles: [1, 2, 3, 4, 5, 6, 7, 8, 9, 
                  10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                  20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 
                  30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 
                  40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                  50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
                  60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
                  70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
                  80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
                  90, 91, 92, 93, 94, 95, 96, 97, 98, 99,
                  100]
              top_n: [25, 50, 100]
          
bias_audit_config:
    from_obj_table: 'data.projects'
    attribute_columns:
      - 'teacher_prefix'
    knowledge_date_column: 'date_posted'
    entity_id_column: 'entity_id'
    ref_groups_method: 'predefined'
    ref_groups:
        'teacher_prefix': 'Mr.'
    thresholds:
        percentiles: [5, 10, 15, 20, 25, 50, 100]
        top_n: [25, 50, 100]

individual_importance:
    methods: [] # empty list means don't calculate individual importances
    n_ranks: 1 
"""

## database.yaml

In [ ]:
database_yaml = """
host: localhost
user: postgres
db: donors_choose
pass: postgres
port: 5432
role: postgres
"""

## run.py

In [ ]:
import yaml

from sqlalchemy.engine.url import URL
from triage.util.db import create_engine
from triage.experiments import MultiCoreExperiment
import logging

import os

from sqlalchemy.event import listens_for
from sqlalchemy.pool import Pool

def run_triage():

  # andrew_id = os.getenv('USER')
  # user_path = os.path.join('/data/users/', andrew_id)
  user_path = '/content'

  # add logging to a file (it will also go to stdout via triage logging config)
  log_filename = os.path.join(user_path, 'triage.log')
  logger = logging.getLogger('')
  hdlr = logging.FileHandler(log_filename)
  hdlr.setLevel(15)   # verbose level
  hdlr.setFormatter(logging.Formatter('%(name)-30s  %(asctime)s %(levelname)10s %(process)6d  %(filename)-24s  %(lineno)4d: %(message)s', '%d/%m/%Y %I:%M:%S %p'))
  logger.addHandler(hdlr)

  # creating database engine
  # dbfile = os.path.join(user_path, 'database.yaml')

  # with open(dbfile, 'r') as dbf:
  #     dbconfig = yaml.safe_load(dbf)

  dbconfig = yaml.safe_load(database_yaml)
  print(dbconfig['role'])

  # assume group role to ensure shared permissions
  @listens_for(Pool, "connect")
  def assume_role(dbapi_con, connection_record):
      logging.debug(f"setting role {dbconfig['role']};")
      dbapi_con.cursor().execute(f"set role {dbconfig['role']};")
      # logging.debug(f"setting role postres;")
      # dbapi_con.cursor().execute(f"set role postgres;")

  db_url = URL(
              'postgres',
              host=dbconfig['host'],
              username=dbconfig['user'],
              database=dbconfig['db'],
              password=dbconfig['pass'],
              port=dbconfig['port'],
          )

  db_engine = create_engine(db_url)

  triage_output_path = os.path.join(user_path, 'triage_output')
  os.makedirs(triage_output_path, exist_ok=True)

  # loading config file
  # with open('%s_triage_config.yaml' % andrew_id, 'r') as fin:
  #     config = yaml.safe_load(fin)

  config = yaml.safe_load(config_yaml)

  # creating experiment object
  experiment = MultiCoreExperiment(
      config = config,
      db_engine = db_engine,
      project_path = triage_output_path,
      n_processes=2,
      n_db_processes=2,
      replace=True,
      save_predictions=True
      )

  # experiment.validate()
  experiment.run()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Let's run triage!

In [ ]:
run_triage()

postgres
2021-10-05 18:48:12 - VERBOSE Matrices and trained models will be saved in /content/triage_output
2021-10-05 18:48:12 -  NOTICE Replace flag is set to true. Matrices, models, evaluations and predictions (if exist) will be replaced
2021-10-05 18:48:12 -    INFO No results_schema_versions table exists, which means that this installation is fresh. Upgrading db.
2021-10-05 18:48:12 -    INFO Context impl PostgresqlImpl.
2021-10-05 18:48:12 -    INFO Will assume transactional DDL.
2021-10-05 18:48:12 -    INFO Running upgrade  -> 8b3f167d0418, empty message
2021-10-05 18:48:12 -    INFO Running upgrade 8b3f167d0418 -> 0d44655e35fd, empty message
2021-10-05 18:48:12 -    INFO Running upgrade 0d44655e35fd -> 264245ddfce2, empty message
2021-10-05 18:48:12 -    INFO Running upgrade 264245ddfce2 -> 72ac5cbdca05, Change importance to float
2021-10-05 18:48:12 -    INFO Running upgrade 72ac5cbdca05 -> 7d57d1cf3429, empty message
2021-10-05 18:48:12 -    INFO Running upgrade 7d57d1cf3429 

FileNotFoundError: ignored

In [ ]:
import yaml
from sqlalchemy.engine.url import URL
from triage.util.db import create_engine
import pandas as pd

dbconfig = yaml.safe_load(database_yaml)
db_url = URL(
            'postgres',
            host=dbconfig['host'],
            username=dbconfig['user'],
            database=dbconfig['db'],
            password=dbconfig['pass'],
            port=dbconfig['port'],
        )

db_engine = create_engine(db_url)

## Checking the results

Confirming we got some models...

In [ ]:
pd.read_sql('SELECT * FROM triage_metadata.models;', db_engine)

,model_id,model_group_id,model_hash,run_time,batch_run_time,model_type,hyperparameters,model_comment,batch_comment,config,train_end_time,test,train_matrix_uuid,training_label_timespan,model_size,random_seed,built_in_triage_run
0,1,3,58b2330d957c967f3e9de50a09eca3da,2021-10-05 18:30:22.081775,2021-10-05 18:30:21.732758,triage.component.catwalk.estimators.classifier...,"{'C': 0.1, 'penalty': 'l1'}",triage demo,None,None,2012-04-01,False,88033d4d2e731553062324d7403af43f,90 days,0.0625,1656233507,1
1,2,2,395896e2f4506e13d4ebe536c72f4d81,2021-10-05 18:30:22.110807,2021-10-05 18:30:21.732758,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 3, 'max_features': None, 'min_sa...",triage demo,None,None,2012-04-01,False,88033d4d2e731553062324d7403af43f,90 days,0.0625,908907174,1
2,3,4,fae0c1a55e7c9bd9fa9064b2cd481371,2021-10-05 18:30:51.265064,2021-10-05 18:30:21.732758,triage.component.catwalk.baselines.rankers.Per...,"{'descend': False, 'feature': 'project_feature...",triage demo,None,None,2012-04-01,False,88033d4d2e731553062324d7403af43f,90 days,0.0625,1259133573,1
3,4,2,232a3978ccf34b167884daf702e8ed26,2021-10-05 18:30:51.883469,2021-10-05 18:30:21.778516,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 3, 'max_features': None, 'min_sa...",triage demo,None,None,2012-08-01,False,10f30faf4d136757ef3f11d9b3e5bdc2,90 days,0.0625,1106414652,1
4,5,3,b21e79f1c30cf98f035f493fd8dae06c,2021-10-05 18:30:59.596706,2021-10-05 18:30:21.778516,triage.component.catwalk.estimators.classifier...,"{'C': 0.1, 'penalty': 'l1'}",triage demo,None,None,2012-08-01,False,10f30faf4d136757ef3f11d9b3e5bdc2,90 days,0.0625,897217774,1
5,6,4,dd806432f4bfd195aade5a559cf076d2,2021-10-05 18:31:25.758532,2021-10-05 18:30:21.778516,triage.component.catwalk.baselines.rankers.Per...,"{'descend': False, 'feature': 'project_feature...",triage demo,None,None,2012-08-01,False,10f30faf4d136757ef3f11d9b3e5bdc2,90 days,0.0625,1431439151,1
6,7,2,4b37802241efcfae1e82485fb3f48ef9,2021-10-05 18:31:35.025329,2021-10-05 18:30:21.817354,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 3, 'max_features': None, 'min_sa...",triage demo,None,None,2012-12-01,False,f8772c9ca6484d2f32a7e8258f6dbc64,90 days,0.0625,1463730397,1
7,8,3,21c8665d4b12f8f095c90dd0bf0d5f68,2021-10-05 18:31:35.416849,2021-10-05 18:30:21.817354,triage.component.catwalk.estimators.classifier...,"{'C': 0.1, 'penalty': 'l1'}",triage demo,None,None,2012-12-01,False,f8772c9ca6484d2f32a7e8258f6dbc64,90 days,0.0625,1879462244,1
8,9,4,3510f611b7ec6e3c4751ba0f81170d63,2021-10-05 18:32:10.612781,2021-10-05 18:30:21.817354,triage.component.catwalk.baselines.rankers.Per...,"{'descend': False, 'feature': 'project_feature...",triage demo,None,None,2012-12-01,False,f8772c9ca6484d2f32a7e8258f6dbc64,90 days,0.0625,827031307,1
9,10,1,afe9d61f915aed6702fc598ceb66fdaf,2021-10-05 18:32:17.412778,2021-10-05 18:30:21.732758,sklearn.ensemble.RandomForestClassifier,"{'max_depth': 50, 'n_estimators': 150, 'min_sa...",triage demo,None,None,2012-04-01,False,88033d4d2e731553062324d7403af43f,90 days,0.0625,1443952767,1


Confirming we saved predictions...

In [ ]:
pd.read_sql('SELECT * FROM test_results.predictions LIMIT 5;', db_engine)

,model_id,entity_id,as_of_date,score,label_value,rank_abs_no_ties,rank_abs_with_ties,rank_pct_no_ties,rank_pct_with_ties,matrix_uuid,test_label_timespan
0,2,247674,2012-05-23,1.0,0,1,1,0.00258,0.25,2a2ba0ccc318bbe74a99726a72c323b0,90 days
1,2,247731,2012-05-22,1.0,0,2,1,0.00515,0.25,2a2ba0ccc318bbe74a99726a72c323b0,90 days
2,2,251302,2012-05-04,1.0,0,3,1,0.00773,0.25,2a2ba0ccc318bbe74a99726a72c323b0,90 days
3,2,256562,2012-04-04,1.0,0,4,1,0.01031,0.25,2a2ba0ccc318bbe74a99726a72c323b0,90 days
4,2,256616,2012-04-04,1.0,0,5,1,0.01289,0.25,2a2ba0ccc318bbe74a99726a72c323b0,90 days


Location of triage outputs...

In [ ]:
!ls triage_output/

matrices  trained_models


In [ ]:
!ls -la triage_output/matrices/


total 128
drwxr-xr-x 2 root root  4096 Oct  5 18:30 .
drwxr-xr-x 4 root root  4096 Oct  5 18:30 ..
-rw-r--r-- 1 root root 17352 Oct  5 18:30 10f30faf4d136757ef3f11d9b3e5bdc2.csv.gz
-rw-r--r-- 1 root root  7029 Oct  5 18:30 10f30faf4d136757ef3f11d9b3e5bdc2.yaml
-rw-r--r-- 1 root root  3950 Oct  5 18:30 2a2ba0ccc318bbe74a99726a72c323b0.csv.gz
-rw-r--r-- 1 root root  3369 Oct  5 18:30 2a2ba0ccc318bbe74a99726a72c323b0.yaml
-rw-r--r-- 1 root root  6136 Oct  5 18:30 76c9a74b579fcdc3747e18fdfa286e35.csv.gz
-rw-r--r-- 1 root root  3345 Oct  5 18:30 76c9a74b579fcdc3747e18fdfa286e35.yaml
-rw-r--r-- 1 root root  8953 Oct  5 18:30 88033d4d2e731553062324d7403af43f.csv.gz
-rw-r--r-- 1 root root  4125 Oct  5 18:30 88033d4d2e731553062324d7403af43f.yaml
-rw-r--r-- 1 root root 10533 Oct  5 18:30 cc681ad650fafd044897af6bd73ab167.csv.gz
-rw-r--r-- 1 root root  3393 Oct  5 18:30 cc681ad650fafd044897af6bd73ab167.yaml
-rw-r--r-- 1 root root 23340 Oct  5 18:30 f8772c9ca6484d2f32a7e8258f6dbc64.csv.gz
-rw-r--r-